# Adding Table Constraints

<hr style="border:2px solid blue">

In [21]:
from sqlalchemy import create_engine, text
import os
import psycopg2
import pandas as pd

In [2]:
# System variables and connection_string

# System Variables
db_name = os.getenv('stock_db')
db_user = os.getenv('db_user')
db_password = os.getenv('db_password')
db_host = os.getenv('db_host')
db_port = os.getenv('db_port')

# For create_engine from sqlalchemy
connection_string = f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

### First table - daily_stock_prices

In [17]:
# Create the SQLAlchemy engine
engine = create_engine(connection_string)
pd.read_sql("""SELECT * FROM daily_stock_prices LIMIT 5;""", engine)

,id,date,symbol,open,close,high,low,volume
0,1,2023-01-03,A,151.960007,150.039993,153.130005,148.470001,1414300
1,2,2023-01-03,AAL,12.910000,12.740000,13.000000,12.530000,21865100
2,3,2023-01-03,AAPL,130.279999,125.070000,130.899994,124.169998,112117500
3,4,2023-01-03,ABBV,162.039993,162.380005,163.020004,160.809998,4937500
4,5,2023-01-03,ABNB,87.385002,84.900002,88.139999,84.300003,3901200


### Second table - sp500_profiles

In [18]:
# Create the SQLAlchemy engine
engine = create_engine(connection_string)
pd.read_sql("""SELECT * FROM sp500_profiles LIMIT 5;""", engine)

,id,asof_date,symbol,company_name,exchange,sector,industry,mktcap,description,ceo,ipo_date,website
0,1,2024-08-24,MMM,3M Company,New York Stock Exchange,Industrials,Conglomerates,71718164700,3M Company operates as a diversified technolog...,Mr. William M. Brown,1946-01-14,https://www.3m.com
1,2,2024-08-24,AOS,A. O. Smith Corporation,New York Stock Exchange,Industrials,Industrial - Machinery,12038155500,A. O. Smith Corporation manufactures and marke...,Mr. Kevin J. Wheeler,1983-09-30,https://www.aosmith.com
2,3,2024-08-24,ABT,Abbott Laboratories,New York Stock Exchange,Healthcare,Medical - Devices,196069331000,"Abbott Laboratories, together with its subsidi...",Mr. Robert B. Ford,1980-03-17,https://www.abbott.com
3,4,2024-08-24,ABBV,AbbVie Inc.,New York Stock Exchange,Healthcare,Drug Manufacturers - General,348940467000,"AbbVie Inc. discovers, develops, manufactures,...",Mr. Robert A. Michael,2013-01-02,https://www.abbvie.com
4,294,2024-08-25,MLM,"Martin Marietta Materials, Inc.",New York Stock Exchange,Basic Materials,Construction Materials,34037335332,"Martin Marietta Materials, Inc., a natural res...",Mr. C. Howard Nye,1994-02-17,https://www.martinmarietta.com


### Altering the table - Adding Foreign Key Constraint

I want to give daily_stock_prices the constraint of a foreign key referencing sp500_profiles. Both tables' fields named "symbol" contain matching values. I will assign this constraint to provide validation of the data in both tables. 

In [22]:
# Using psycopg2 to create a constraint of a foreign key using stock symbols shared in both tables

connection = psycopg2.connect(
    dbname=db_name,
    host=db_host,
    user=db_user,
    port=db_port,
    password=db_password)

# Constraint addition
try:
    # Execute the SQL command to add the foreign key constraint
    with connection.cursor() as cursor:
        cursor.execute("""
        ALTER TABLE daily_stock_prices
        ADD CONSTRAINT fk_symbol
        FOREIGN KEY (symbol)
        REFERENCES sp500_profiles(symbol);
        """)
    
    # Commit the changes to the database
    connection.commit()
    print("Foreign key constraint added successfully.")

# If there is an erro
except Exception as e:
    # Rollback in case of an error
    connection.rollback()
    print(f"An error occurred: {e}")

# Close the connection to the db
finally:
    connection.close()

Foreign key constraint added successfully.


In [19]:
# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Execute raw SQL command to add a foreign key constraint
with engine.connect() as conn:
    conn.execute(text("""
    ALTER TABLE daily_stock_prices
    ADD CONSTRAINT fk_symbol
    FOREIGN KEY (symbol)
    REFERENCES sp500_profiles(symbol);
    """))

<hr style="border:2px solid blue">

### Verification - using sqlalchemy and pandas

In [23]:
# fetchall() to check constraints

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

# Execute raw SQL command to query constraints
with engine.connect() as conn:
    result = conn.execute(text("""
    SELECT constraint_name
    FROM information_schema.table_constraints
    WHERE table_name = 'daily_stock_prices'
    AND constraint_type = 'FOREIGN KEY';
    """))
    constraints = result.fetchall()

# Print out the constraint names
for constraint in constraints:
    print(f"Constraint Name: {constraint[0]}")

Constraint Name: fk_symbol


In [24]:
# Query to get all constraints from daily_stock_prices

df = pd.read_sql("""
SELECT constraint_name, constraint_type
FROM information_schema.table_constraints
WHERE table_name = 'daily_stock_prices';
""", engine)

df

,constraint_name,constraint_type
0,daily_stock_prices_pkey,PRIMARY KEY
1,fk_symbol,FOREIGN KEY
2,2200_16458_1_not_null,CHECK
3,2200_16458_2_not_null,CHECK
4,2200_16458_3_not_null,CHECK
5,2200_16458_4_not_null,CHECK
6,2200_16458_5_not_null,CHECK
7,2200_16458_6_not_null,CHECK
8,2200_16458_7_not_null,CHECK


In [26]:
# Joining 3 schema tables to verify the foreign key assignment where table_name = 'daily_stock_prices' and
# constraint_type = 'FOREIGN KEY'

pd.read_sql("""
SELECT 
    tc.constraint_name, 
    tc.constraint_type,
    kcu.column_name AS fk_column,
    ccu.table_name AS referenced_table,
    ccu.column_name AS referenced_column
FROM 
    information_schema.table_constraints AS tc
    JOIN information_schema.key_column_usage AS kcu
    ON tc.constraint_name = kcu.constraint_name
    JOIN information_schema.constraint_column_usage AS ccu
    ON ccu.constraint_name = tc.constraint_name
WHERE 
    tc.table_name = 'daily_stock_prices'
    AND tc.constraint_type = 'FOREIGN KEY';""", engine)

,constraint_name,constraint_type,fk_column,referenced_table,referenced_column
0,fk_symbol,FOREIGN KEY,symbol,sp500_profiles,symbol
